# Run Models on 2019 Data Joined with Incarceration Rates
#### 10/23/2020
---

## Load and Process Dataset
### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import wandb
import warnings
warnings.filterwarnings('ignore')

In [2]:
wandb.init(project="2019_newvar_incarceration")

wandb: Currently logged in as: apraturu (use `wandb login --relogin` to force relogin)


### Read in `adult.csv` data

In [3]:
df = pd.read_csv('../final_data/2019_new-var_inc.csv').sample(frac=0.5)
df.head()

,Geographic-division,area-code,region,state,RT,person-weight,age,citizenship-status,class-worker,education,...,Incarceration rate per 100000,Incarceration rate: White alone,Incarceration rate: Black or African American alone,Incarceration rate: American Indian and Alaska Native alone,Incarceration rate: Asian alone,Incarceration rate: Native Hawaiian and other Pacific Islander alone,Incarceration rate: Some other race alone,Incarceration rate: Two or more races,Incarceration rate: Hispanic or Latino,"Incarceration rate: White alone, not Hispanic or Latino"
575460,East North Central,2100,Midwest,IN,P,557,69,Born-US,Self-emp-not-inc,some-college-great1,...,751.009978,563.579549,2814.353133,888.311126,75.141011,553.662692,434.471537,177.481020,781.356250,542.121532
1355934,East South Central,2504,South,TN,P,179,45,Born-US,Private-prof,some-college-great1,...,739.934180,509.026101,1962.139949,665.199560,112.886609,137.287205,441.689268,262.705779,594.706594,502.459808
409301,South Atlantic,903,South,FL,P,109,44,Born-US,Private-prof,GED,...,890.645386,610.787515,2554.784187,590.556691,98.720156,293.016441,353.522897,279.531166,536.009466,626.024257
756402,New England,1901,Northeast,MA,P,91,19,Born-US,Private-prof,some-college-less1,...,376.976154,310.660339,1502.308942,981.432361,64.328355,719.748088,387.349214,111.625960,927.581119,241.032739
1134019,South Atlantic,4800,South,NC,P,90,72,Born-US,Private-prof,HSgrad,...,646.847150,376.752770,1664.479837,882.810581,105.282300,272.562084,316.402193,173.133720,490.676399,356.796559


In [4]:
df.loc[df['income'] <= 50000, 'income_adjusted'] = '<=50K'
df.loc[df['income'] > 50000, 'income_adjusted'] = '>50K' 

df = df.drop(columns=['income']).rename(columns={'income_adjusted': 'income'}).dropna()

df.head()

,Geographic-division,area-code,region,state,RT,person-weight,age,citizenship-status,class-worker,education,...,Incarceration rate: White alone,Incarceration rate: Black or African American alone,Incarceration rate: American Indian and Alaska Native alone,Incarceration rate: Asian alone,Incarceration rate: Native Hawaiian and other Pacific Islander alone,Incarceration rate: Some other race alone,Incarceration rate: Two or more races,Incarceration rate: Hispanic or Latino,"Incarceration rate: White alone, not Hispanic or Latino",income
172780,Pacific,1302,West,CA,P,41,45,Born-US,Private-prof,prof-school,...,605.996084,3036.268547,996.414012,94.424879,617.788428,703.884463,188.775488,756.679936,453.449136,>50K
1417654,West South Central,3501,South,TX,P,43,35,Born-US,Fed-gov,master,...,843.762174,2854.646835,584.891093,172.611124,323.236055,1079.147366,278.644656,971.734147,768.328062,>50K
383288,South Atlantic,8604,South,FL,P,45,28,Born-abroad-US-parents,Private-prof,prof-school,...,610.787515,2554.784187,590.556691,98.720156,293.016441,353.522897,279.531166,536.009466,626.024257,<=50K
208105,Pacific,3713,West,CA,P,43,53,not-US-citizen,Local-gov,master,...,605.996084,3036.268547,996.414012,94.424879,617.788428,703.884463,188.775488,756.679936,453.449136,>50K
452896,South Atlantic,4006,South,GA,P,105,47,Born-US,Fed-gov,doctorate,...,681.372075,2036.242113,581.630431,120.521390,603.029857,808.492254,360.982992,1098.995067,639.665898,>50K


In [5]:
df.isnull().sum()

Geographic-division                                                     0
area-code                                                               0
region                                                                  0
state                                                                   0
RT                                                                      0
person-weight                                                           0
age                                                                     0
citizenship-status                                                      0
class-worker                                                            0
education                                                               0
marital-status                                                          0
occupation                                                              0
relationship                                                            0
race                                  

In [6]:
X = df.drop(columns=['income'], axis = 1)
y = df[['income']].to_numpy()

In [7]:
labels = np.unique(y)
features = list(X.columns)

In [15]:
X.columns

Index(['Geographic-division', 'area-code', 'region', 'state', 'RT',
       'person-weight', 'age', 'citizenship-status', 'class-worker',
       'education', 'marital-status', 'occupation', 'relationship', 'race',
       'sex', 'extra-income', 'hours-per-week', 'field-of-degree',
       'place-of-birth', 'stem-degree', 'Incarceration rate per 100000',
       'Incarceration rate: White alone',
       'Incarceration rate: Black or African American alone',
       'Incarceration rate: American Indian and Alaska Native alone',
       'Incarceration rate: Asian alone',
       'Incarceration rate: Native Hawaiian and other Pacific Islander alone',
       'Incarceration rate: Some other race alone',
       'Incarceration rate: Two or more races',
       'Incarceration rate: Hispanic or Latino',
       'Incarceration rate: White alone, not Hispanic or Latino'],
      dtype='object')

### Pre-process categorical variables

In [8]:
cat_var = ['Geographic-division', 'area-code', 'region', 'state', 'RT', 'citizenship-status', 'class-worker', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'field-of-degree', 'place-of-birth', 'stem-degree']
for f in cat_var:
    enc = preprocessing.LabelEncoder()
    X[f] = enc.fit_transform(X[f])

scaler = preprocessing.StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

### Split data into test and training

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
len(X_test)

61523

## Train Models and Establish Baseline Scores

### Random Forest

In [11]:
# Train model, get predictions
model = RandomForestClassifier()
model.fit(X, y)
y_pred = model.predict(X_test)
y_probas = model.predict_proba(X_test)
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

In [12]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246091 entries, 294269 to 10909
Data columns (total 30 columns):
 #   Column                                                                Non-Null Count   Dtype  
---  ------                                                                --------------   -----  
 0   Geographic-division                                                   246091 non-null  float64
 1   area-code                                                             246091 non-null  float64
 2   region                                                                246091 non-null  float64
 3   state                                                                 246091 non-null  float64
 4   RT                                                                    246091 non-null  float64
 5   person-weight                                                         246091 non-null  float64
 6   age                                                                   246091 non

In [13]:
# Visualize model performance
wandb.sklearn.plot_classifier(
    model, X_train, X_test, y_train, y_test, y_pred, y_probas, labels,
    is_binary=True, model_name='RandomForest', feature_names=features)

wandb: 
wandb: Plotting RandomForest.
wandb: Logged feature importances.
wandb: Logged learning curve.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision recall curve.
